###### The cell below loads the visual style of the notebook when run.

In [80]:
from IPython.core.display import HTML
css_file = '../../styles/styles.css'
HTML(open(css_file, "r").read())

# Fitting an isochrone to your data



<section class="objectives panel panel-warning">
<div class="panel-heading">
<h2><span class="fa fa-certificate"></span>Learning Objectives</h2>
</div>
</section>

> * Understand the location of co-eval stars in the HR diagram, and how it changes with age.
> * Learn how to use the ```isochrones``` library to create a model of co-eval stars.
> * Compare stellar models to your own open cluster data to find the best-fitting age and errors.


## Theory

We work on the assumption that all the stars in a cluster are born at the same time (are *co-eval*), and have the same metallicity. Therefore, the main factor that accounts for the different properties of the individual stars is that they have different masses. 

Having produced a colour-magnitude diagram of our open cluster in previous lab sessions, in this session we compare to stellar models in order to calculate the age, distance and [interstellar extinction](http://slittlefair.staff.shef.ac.uk/teaching/phy241/lectures/L03/index.html) of the open cluster.

<section class="challenge panel panel-success"> 
<div class="panel-heading">
<h2><span class="fa fa-pencil"></span> Read in your data</h2>
</div>
</section>

> At the end of the previous session, you should have created a CSV file containing apparent $V$ magnitudes, and $B-V$ colours for all of the stars in your open cluster. 

> Upload that file to somewhere on CoCalc (probably easiest to load into the same folder as this notebook). If you haven't finished the previous session yet, use the example CSV file
> included in this assignment ```example_data.csv```, which has only two columns ($V$ and $B-V$)

> Write some code that reads in the CSV file into two arrays - one for the $V$ magnitudes and one for the $B-V$ colours.

In [81]:
import matplotlib.pyplot as plt
import numpy as np
from numpy import genfromtxt


file300 = "../../observing_project/calibrated_data_300.csv"
file30 = "../../observing_project/calibrated_data_30.csv"

V300 = genfromtxt(file300, delimiter=',')
V30 = genfromtxt(file30, delimiter=',')

V_mag = V300[:,0]
B_minus_V_mag = V300[:,1]
B_minus_V_error = V300[:,2]

V_mag30 = V30[:,0]
B_minus_V_mag30 = V30[:,1]
B_minus_V_error30 = V30[:,2]

<section class="challenge panel panel-success"> 
<div class="panel-heading">
<h2><span class="fa fa-pencil"></span> Plot your colour-magnitude diagram</h2>
</div>
</section>

> Write a function that takes these two arrays as the arguments (inputs) and plots a colour-magnitude diagram ($V$ magnitude on the y-axis, $B-V$ on the x-axis). You will want to invert the y-axis so that brighter stars are at the top. You can use the matplotlib function ```axis.invert_yaxis()``` to do this.

> For use later on, your function will want to **return the matplotlib axis** that is created inside the function.

> Use your function to plot the colour-magnitude diagram (CMD).



In [82]:
def plotCMD(Vmag, B_minus_V_mag, error):
    fig, axis = plt.subplots()

    axis.errorbar(B_minus_V_mag, Vmag, xerr=error, fmt="o", ecolor="r")
    axis.set_xlim(-0.5, 2)
    axis.set_ylim(10,17)
    axis.invert_yaxis()
    axis.set_xlabel("B-V")
    axis.set_ylabel("V")
    axis.set_title("Colour Magnitude diagram of NGC 7789")
    axis.grid()
    
    return axis

In [83]:
CMD300_axis = plotCMD(V_mag, B_minus_V_mag, B_minus_V_error)
CMD300_axis.plot(V_mag, B_minus_V_mag,B_minus_V_error, ".")

In [84]:
CMD300_axis.plot(V_mag, B_minus_V_mag)

In [85]:
CMD300_axis

<Axes: title={'center': 'Colour Magnitude diagram of NGC 7789'}, xlabel='B-V', ylabel='V'>

## The `isochrones` library

The `isochrones` library is not built into Python, but it is installed on the Python3 (System-Wide) kernel on CoCalc. If you are running on your own laptop, you will have to install it yourself (`pip install isochrones`). The `isochrones` library allows you to compute the location in a colour-magnitude diagram of a co-eval population of stars, based on pre-computed grids of evolutionary stellar models.

You can use several sets of stellar models with the library, but we will use the so-called "MIST" stellar models, detailed in [Choi et al 2016](https://ui.adsabs.harvard.edu/abs/2016ApJ...823..102C/abstract).

You can import the MIST Isochrones using the following code:

In [86]:
from isochrones import get_ichrone

Once we've imported the library, you can create an isochrone object by specifying the filters you are interested in, like so:

In [87]:
iso = get_ichrone('mist', bands=['Bessell_B', 'Bessell_V'])

Here we've stored an *isochrone object* in the variable named  `iso`. This object contains a single function `isochrone`, that allows us to calculate the CMD location of stars of a given age. The single argument of this function is $\log_{10}$ of the age in years, so, for a 1.7 Gyr cluster we would use:

In [88]:
model = iso.isochrone(9.235)  # note the argument is log10(1.7e9)
print(type(model))

<class 'pandas.core.frame.DataFrame'>


The `isochrone` function returns a `DataFrame` object from the `pandas` library. We haven't time or space to go into the `pandas` library in this course, except to say that many people love it for dealing with  data tables. For now, all we need to know is we can access the theoretical $B$ and $V$ magnitudes like so:

In [89]:
model_b = model.Bessell_B_mag
model_v = model.Bessell_V_mag
# calculate B-V for this model
model_bv = model_b - model_v

<section class="challenge panel panel-success"> 
<div class="panel-heading">
<h2><span class="fa fa-pencil"></span> Plot the isochrone</h2>
</div>
</section>

> Using the isochrones library, and your function from earlier, plot an isochrone on your HR diagram. Use an isochrone of approximately the correct age for your cluster.


In [90]:
iso.isochrone?

Signature:
iso.isochrone(
    age,
    feh=0.0,
    eep_range=None,
    distance=10.0,
    AV=0.0,
    dropna=True,
)
Docstring: <no docstring>
File:      /usr/local/lib/python3.10/dist-packages/isochrones/models.py
Type:      method

In [91]:
NGC7789_model = iso.isochrone(9.204)  # note the argument is log10(1.7e9)
model_b = NGC7789_model.Bessell_B_mag
model_v = NGC7789_model.Bessell_V_mag

# calculate B-V for this model
model_bv = model_b - model_v

#Create axis variable 
CMD300_axis = plotCMD(V_mag, B_minus_V_mag, B_minus_V_error)

#Set y limit so isochrone is visible 
CMD300_axis.set_ylim(0,17)

#Invert y axis 
CMD300_axis.invert_yaxis()

#plot isochrone on CMD 
plt.plot(model_v, model_bv)

# Distance, extinction and reddening

If you've done the last step correctly, you should find the isochrone lies nowhere near your data. What's going on?

The answer is that your data are in **apparent** magnitudes, **and** the light from the stars has been extincted and reddened by dust lying between us and the cluster. The `isochrones` library returns **absolute**, unreddened magnitudes.

The conversion between absolute and apparent magnitudes depends upon the distance to the cluster:

$$m - M = 5 \log_{10} (d/10),$$

where $d$ is the distance in parsecs. Notice that this will affect the magnitudes, but not the colour! This means the effect of distance is to move the isochrone vertically on the CMD.

What about reddening and extinction? Just like extinction by dust in our own atmosphere, interstellar extinction will make the stars fainter, and redder. So, whilst a finite distance moves an star vertically in the CMD, dust along the line of sight will move a star down and to the right.

We can write that the total extinction in the $V$-band, in magnitudes, is $A_V$. This means that if the true magnitude of a star is $V_0$, the observed magnitude is $V = V_0 + A_V$. However, the dust also makes the star redder, so we can write that the true colour of the star is related to the observed colour is $(B-V) = (B-V)_0 + E(B-V)$. The quantity $E(B-V)$ is known as the colour excess, or reddening.

Since both $A_V$ and $E(B-V)$ are related to the amount of dust between us and the cluster, it should not surprise you that they are related. It turns out that $A_V = 3.1 E(B-V)$; see [Shultz & Wiemer (1975)](https://ui.adsabs.harvard.edu/abs/1975A%26A....43..133S/abstract), for example.

<section class="challenge panel panel-success"> 
<div class="panel-heading">
<h2><span class="fa fa-pencil"></span> Apply distance and reddening</h2>
</div>
</section>

> Write a new function that accepts four arguments: a matplotlib axis, the logarithm of the age, the distance and the extinction, and plots an isochrone on the provided axis. Correct the V magnitudes and the B-V colours of the isochrone for distance and reddening. I've provided a template to get you started.

> Use your function in combination with others you've plotted above to plot your CMD with three isochrones on top of it. Plot one isochrone of approximately the correct age, distance and reddening. Then plot two more; one a Gyr older, the other a Gyr younger.

In [153]:
help(iso.isochrone)

Help on method isochrone in module isochrones.models:

isochrone(age, feh=0.0, eep_range=None, distance=10.0, AV=0.0, dropna=True) method of isochrones.mist.isochrone.MIST_Isochrone instance



In [158]:
def plot_isochrone(axis, log_age, distance, a_v, legend_label):
    """
    Plots a MIST group isochrone on the axis supplied by the user

    Parameters
    ----------
    axis: matplotlib.axis
        the matplotlib axis object on which we want to plot
    log_age: float
        logarithm of the age in years (base 10)
    distance: float
        distance to cluster in parsecs
    a_v: float
        V-band extinction to cluster, in magnitudes
    legend_label: String
        Give a label for the isochrone in the legend
    """
    model = iso.isochrone(age=log_age, distance=distance, AV=a_v)
    b_model = model.Bessell_B_mag
    v_model = model.Bessell_V_mag

    model_BV = b_model - v_model

    axis.plot(model_BV, v_model, zorder=10, label=f"{legend_label}, distance: {distance} pc, Reddening value: {a_v:.2f}:")
    axis.legend()

In [159]:
age = np.log10(1.6e9) # log_10(1.6e9) NGC 7789 age is around 1.6e9 Gyr old
age_younger = np.log10(0.6e9) # 1Gyr younger
age_older = np.log10(2.6e9) # 1 Gyr older 

distance = 1870 # Distance to NGC7789 in Parsecs
extinction = 0.25

CMD300_axis = plotCMD(V_mag, B_minus_V_mag, B_minus_V_error)
#CMD300_axis.plot(V_mag30, B_minus_V_mag30,B_minus_V_error30, ".")

plot_isochrone(CMD300_axis, age, distance, extinction, f"age: {age:.3f}")

#Plot isochrone of 1 Gyr older
plot_isochrone(CMD300_axis, age_older, distance, extinction, f"age: {age_older:.3f}")

#Plot isochrone of 1 Gyr younger
plot_isochrone(CMD300_axis, age_younger, distance, extinction, f"age: {age_older:.3f}")

---------
<section class="challenge panel panel-success"> 
<div class="panel-heading">
<h1>Homework #7</h1>
<h2><span class="fa fa-pencil"></span>  Find the best fitting isochrone</h2>
</div>
</section>

Now it's time to use the code you wrote above to fit the isochrones to your data and find the best fitting model. Follow the instructions below to find the age, distance and reddening to your cluster, with errors.

By trial and error, using the functions written above, you can alter the distance, reddening and age until the isochrone sequence lines up with the data.

However, if you **only fit the position of the main-sequence stars**, you could do this for an isochrone of **any** age! Even if the isochrone is the wrong age there will be a value of the reddening and distance which will make the main sequence of the isochrone line up with the main sequence in the data. How then do we find the age of the cluster?

The answer is that only an isochrone of the correct age will simultaneously fit the main sequence, the location of the M-S turn-off and the location of giant stars in your cluster. If you've chosen the right age, the location of stars in various evolutionary stages will line up with the isochrone - if you have the right distance and reddening.

You might want to read the companion notebook `understanding_CMDs.ipynb`, which helps explain where stars at various stages of evolution appear in the CMD.

To put all this together, you can imagine finding the age, distance and reddening all from one colour magnitude diagram. You would follow a process something like this:

1. Pick an age. 
2. Adjust the distance and reddening until the isochrone lies near the data. 
3. Try a slightly younger, or older age and repeat step 2.
4. If the fit has got worse, try changing the age in the other direction.

When you find the right age, distance and redenning, the isochrone should match up with the data!

<section class="challenge panel panel-success"> 
<div class="panel-heading">
<h2><span class="fa fa-question"></span>  Q1: Find the best fitting isochrone (4 points)</h2>
</div>
</section>

> Using the method above, find the distance, reddening and age that best fits your cluster. Make a plot of the best fitting age.

In [160]:
age = np.log10(1.6e9) # log_10(1.6e9) NGC 7789 age is around 1.6e9 Gyr old


age1= 1.55e9
log_age1=np.log10(age1)

age2 = 1.7e9
log_age2 = np.log10(age2)

age3 = 2.1e9
log_age3 = np.log10(age3)

#distance = 1870 # Distance to NGC7789 in Parsecs
extinction = 0.75


CMD300_axis = plotCMD(V_mag, B_minus_V_mag, B_minus_V_error)
CMD300_axis.set_ylim(11,17)
CMD300_axis.invert_yaxis()

#plot different ages, with the same distance and reddening
plot_isochrone(CMD300_axis, log_age1, 1900, extinction, f"age: {log_age1:.3f}") 
plot_isochrone(CMD300_axis, log_age2, 1900, extinction, f"age: {log_age2:.3f}")
plot_isochrone(CMD300_axis, log_age3, 1900, extinction, f"age: {log_age3:.3f}")

# Age of 1.55e9 fits best 

In [138]:
CMD300_axis = plotCMD(V_mag, B_minus_V_mag, B_minus_V_error)
CMD300_axis.set_ylim(11,17)
CMD300_axis.invert_yaxis()

age1= 1.55e9
log_age1=np.log10(age1)

#Plot best fitting age and change distance and reddening values
plot_isochrone(CMD300_axis, log_age1, 1850, 0.75, f"age; {log_age3:.3f}") #
plot_isochrone(CMD300_axis, log_age1, 1900, 0.75, f"age; {log_age3:.3f}") #
plot_isochrone(CMD300_axis, log_age1, 1950, 0.75, f"age; {log_age3:.3f}") #

# Distance of 1850 PC fits best 
# Reddening of 0.75 fits bets 

In [161]:
#plot of best fitting model 
CMD300_axis = plotCMD(V_mag, B_minus_V_mag, B_minus_V_error)
CMD300_axis.set_ylim(11,17)
CMD300_axis.invert_yaxis()

#age = 1.6e9
age = 1.55e9
log_age = np.log10(age) 
#distance = 2300 # In Parsecs 1800

distance = 1850
extinction = 0.80


plot_isochrone(CMD300_axis, log_age, distance, extinction, f"Age: {log_age:.3f}")

## Uncertainties

How to judge our uncertainties in the three quantities? One way is to try out isochrones of different ages. Suppose we try an isochrone which is 200 Myr older than our best fit above. We'd need to change the distance and extinction again to get it as close as possible to the data, but once we'd done that, it might still be a "reasonable" fit; i.e it looks OK by-eye. 

What if we tried one 400 Myr older? Perhaps, even after tweaking the distance and reddening, we still had a poor fit to the data, again judged by-eye. In this case, we'd conclude that a cluster 200 Myr older was consistent with our data, but one 400 Myr older is not. Therefore, an uncertainty of 200 Myr on the age would seem reasonable.

We could use the spreads in distance and extinction we found to estimate uncertainties in those properties, too.

<section class="challenge panel panel-success"> 
<div class="panel-heading">
<h2><span class="fa fa-question"></span>  Q2: Finding our errors (4 points)</h2>
</div>
</section>

> Use the approach above to find error bars for your age, distance and extinction to the cluster. Make a plot that has your best fitting isochrone on it, along with the isochrones that represent the worst fit you think is still OK. Clearly label which is which.

In [179]:
CMD300_axis = plotCMD(V_mag, B_minus_V_mag, B_minus_V_error)
CMD300_axis.set_ylim(11,17)
CMD300_axis.invert_yaxis()

#Best fitting age
best_age = 1.55e9
#Younger and older ages
age_younger = 1.15e9
age_older = 2.15e9 

#Log of all ages
log_age = np.log10(best_age)
log_age_younger = np.log10(age_younger)
log_age_older = np.log10(age_older)

print(log_age)
distance = 1850 # In Parsecs
extinction = 0.8

plot_isochrone(CMD300_axis, log_age_younger, distance, extinction, "Younger Uncertainty")
plot_isochrone(CMD300_axis, log_age, distance, extinction, "Best fitting")
plot_isochrone(CMD300_axis, log_age_older, distance, extinction, "Older uncertainty")

# For best fitting younger and older models
# need to adust distance by +325 pc and extinction by +0.08 for 400 Myr younger
# need to adust distance by -310 pc and extinction by -0.06 for 500 Myr older

9.190331698170292


<section class="challenge panel panel-success"> 
<div class="panel-heading">
<h2><span class="fa fa-question"></span> Q3: Systematic Errors (2 points)</h2>
</div>
</section>

> The technique above will give you a good idea of your statistical errors; i.e how the scatter in the CMD affects your ability to accurately judge the age. However, there may be sources of *systematic* error. A systematic error would pull **all the stars** around in the CMD **in the same direction** and therefore systematically affect the age, distance or extinction you derive.

> In the markdown cell below, briefly outline what these may be, and what effect they would have. 

> Hint: you will want to think carefully about the steps you carried out to calculate the absolute photometry of the cluster.

A simpler absolute photometry method was used that doesn't take into account secondary effects, like  colour terms and 2nd order extinction. This could introduce inaccuracies of a few percent more when compared to a more precise calibration method that does take these into account. We also use sky surveys that have accuracies of a few percent, but will also suffer from our ignorance of secondary effects. 

When we calculate the background statistics, such as median and mean of sky background, we specify a threshold for the DAOFIND algorithm. If the threshold is too large or too small this can introduce potential inaccuracies 

When creating the source list and plotting the sources , we give an aperture radius, if this is too small or too large then this could introduce problems such as light contamination from nearby stars, or detecting objects that are not stars \(for example background pixels\)

When applying our aperture photometry method we specify an aperture radius and inner and outer annulus radii, these can have a similar effect as above, as well as introducing noise if any of the radii are too large
